In [74]:
### fidelity 계산 부분이랑 99.99이상이면 에이스타 한번 더 안 돌리는거랑 - 완료
### 애초에 모델에서 나온 값을 저장할 때 fidelity도 같이 계산해서 csv파일에 같이 저장해야 하고 - 완료
### 혹시 fid 떨어지는 애 있는지 검사하는 부분도 추가해야함 - ?
### main에서 초기화 할 때 comb 어떻게 넣어야 할지 봐야 함 - 완료
### DL 모델 코드에서 fid 항목 내보내기 필요 - 완료

In [75]:
# Standard libraries
from datetime import datetime
import heapq
import os
import random
import time

# Numeric and mathematical libraries
from math import *
import numpy as np
from scipy.linalg import expm, fractional_matrix_power

# Data analysis and processing libraries
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Visualization libraries
import matplotlib.pyplot as plt

# Quantum physics and related libraries
from qutip import *

In [76]:
# pauli matrix 
sx = np.array([[0,  1],     [1, 0]])
sy = np.array([[0, -1j],   [1j, 0]])
sz = np.array([[1, 0],     [0, -1]])
s0 = np.array([[1, 0],      [0, 1]])

# parameters(detuning factor)
d0 = 0.15           # Arbitrary settings, Actual speed : 0.30rad/μs
dt = 1.3
v0 = 0.02           # Arbitrary settings, Actual speed : 0.04rad/μs

# Initial state
init_wave = np.array([[1], [0]])
irho_init = np.kron(init_wave, init_wave.conj().T)


# unitary operator
def unitary(dt, choice):
    
    # Select x,y-rotation direction.
    # [stay, +x, -x, +y, -y]
    choice_list = [0, 1, -1, 1, -1] 
    
    if choice < 3:
        # if choice = 0 ... only d0*sz
        Ham = (d0*sz+v0*choice_list[choice]*sx)
    else:
        Ham = (d0*sz+v0*choice_list[choice]*sy)

    # Creating a Unitary Operator for each of the four sections by Hamiltonian
    eigvals = np.linalg.eigh(Ham)[0]
    eigvecs = 1*np.linalg.eigh(Ham)[1]
    E = np.diag(eigvals)
    U_H = eigvecs.conj().T
    U_e = U_H.conj().T @ expm(-1j*E*dt) @ U_H
    
    return U_e


#######################################

# Set unitary choicelist as global variable 
#   because unitary operators are used repeatedly.

unitary_choiceList = [unitary(dt, i) for i in range(5)]
    
#######################################

# x-rotation operater
def Rx(theta):
    return np.matrix([  [cos(theta/2),    -1j*sin(theta/2)],
                        [-1j*sin(theta/2),    cos(theta/2)]])

# z-rotation operater
# Do not use Rz. Control by rotation only by Hamiltonian.
def Rz(phi): 
    return np.matrix([  [cos(phi/2)-1j*sin(phi/2),  0],
                        [0,  cos(phi/2)+1j*sin(phi/2)]])


# Calculating the Fidelity
def state_fidelity(rho_1, rho_2): 
    
    # rho_1(current state), rho_2(target state)
    # Calculate the fidelity after checking the dimensions of the two states.
    
    if np.shape(rho_1) != np.shape(rho_2):
            print("Dimensions of two states do not match.")
            return 0
    else:
        sqrt_rho_1 = fractional_matrix_power(rho_1, 1 / 2)
        fidelity = np.trace(fractional_matrix_power(sqrt_rho_1 @ rho_2 @ sqrt_rho_1, 1 / 2)) ** 2
        return np.real(fidelity)
    

def fidelity(target_theta, target_phi, predicted_sequence):
    
    target_U = Rz(target_phi) @ Rx(target_theta)
    irho_target = target_U @ irho_init @ target_U.conj().T

    predicted_sequence_indices = np.argmax(predicted_sequence, axis=-1)

    Uni = s0
    U_0 = unitary(dt, 0)
    U_1 = unitary(dt, 1)
    U_2 = unitary(dt, 2)
    U_3 = unitary(dt, 3)
    U_4 = unitary(dt, 4)
    
    pulse_list = [U_0, U_1, U_2, U_3, U_4]
    
    for i in predicted_sequence_indices[0]:
        Uni = pulse_list[i] @ Uni

    irho_final = Uni @ irho_init @ Uni.conj().T
    
    F = (state_fidelity(irho_final,irho_target))
    
    return  F


# Caculating the Phase
def phase_cal(x, y):
    
    # Calculate phase from quadrant point of view.
    
    if y >= 0 and x >= 0:
        phase = atan(y/x)
    elif y > 0 and x < 0:
        phase = atan(-x/y) + pi/2
    elif y < 0 and x < 0:
        phase = atan(y/x) + pi
    else: 
        phase = atan(-x/y) + 3/2*pi
        
    return phase

In [77]:
class Node:
    
    def __init__(self, parent=None, density_matrix=None, past_pulse=None):
        self.parent = parent
        self.density_matrix = density_matrix
        self.past_pulse = past_pulse

        self.x = round(np.trace(self.density_matrix*sx).real, 9)
        self.y = round(np.trace(self.density_matrix*sy).real, 9)
        self.z = round(np.trace(self.density_matrix*sz).real, 9)
        
        self.phase = phase_cal(self.x, self.y)
        self.point = (self.x, self.y, self.z)
        
        self.g = 0      # Time moved to the current position 
        self.h = 0      # Estimated time to target (heuristic)
        self.f = 0      # f = g + h


# Using Astar Algorithm
def Astar_Qsearch(init, target):
    
    # Precautions 
    # Originally, we can determine whether A* for discrete nodes is the same point or not, 
    #   but in the current system, the likelihood of the same point is almost zero.
    # Points in similar positions should be recognized as the same points to increase search efficiency.
    
 
    # Initialize openList, closedList.
    openList = []
    closedList = []

    # Add initial node to openList.
    heapq.heappush(openList, (init.f, init))

    # Open List Length Maximum Limit
    maximum_len = 100000
    
    # Run until fidelity 0.99 is satisfied.
    iteration = 0
    while openList:

        # In the beginning, even if you rotate, you can't move a lot, 
        #   so just in case you recognize all of those points as the same point, 
        #   you recognize that they are the same only when the fidelity of the two points is 0.99999 or higher.
        # After the specific iteration, it is recognized that the fidelity 
        #   between the points is equal if it is 0.99 or higher.
        # As a strategy to acquire a lot of data, give up if you don't find the right answer within iteration.
        if iteration < 10: 
            fid = 0.99999
        elif iteration < 3000:
            fid = 0.999
        else :
            return [-1] 

        # Get the node with the lowest f-score from the openList.
        _, currentNode = heapq.heappop(openList)

        # Add currentNode to closedList
        closedList.append(currentNode)
        
        # Verifying Code Progress
        print(iteration, " = > ", currentNode.g,currentNode.h,currentNode.f)
        
        # Save and return path if fidelity 0.99 or higher.
        if state_fidelity(currentNode.density_matrix, target.density_matrix) > 0.9999:
            path = []
            current = currentNode
            
            while current is not None:
                path.append(current.past_pulse)
                current = current.parent
                
            path = path[:len(path) - 1]
            path = path[::-1]
            
            # Verifying Code Progress
            print(iteration)
            
            return path
        else:
            print("iter :", iteration, " =>", state_fidelity(currentNode.density_matrix, target.density_matrix))

        children = []
        i = 0
        # Generating children according to rotation
        for rotation in unitary_choiceList:
            NewDensityMatrix = rotation @ currentNode.density_matrix @ rotation.conj().T
            new_node = Node(currentNode, NewDensityMatrix, i)
            children.append(new_node)
            i += 1
        
        # Comparison by Child Case
        for child in children:
            
            # If the child(closedNode) is in the closedList with certain conditions, continue.
            if any(state_fidelity(child.density_matrix, closedNode.density_matrix) > fid for closedNode in closedList):
                print("pass")
                continue
            
            # Update g,h,f values of child
            child.g = currentNode.g + dt
            child.h = heuristic(child, target)
            child.f = child.g + child.h

            # If the child(openNode) is in the openList with certain conditions, continue.
            if any(
                child.f < openNode[0] and 
                child.g > openNode[1].g and
                state_fidelity(child.density_matrix, openNode[1].density_matrix) > 0.999
                for openNode in openList
            ):
                print("pass2")
                continue
            
            # Add the child('s f) to the openList
            heapq.heappush(openList, (child.f, child))
            
        iteration += 1
        
        # Open list cut if open list maximum length limit is exceeded.
        if len(openList) > maximum_len:
            while len(openList) > maximum_len:
                heapq.heappop(openList)

    return [-1]  # No path found

# Heuristic estimation
def heuristic(node, target):
    
    # Time Remaining from Current Node to Target
    # Consider both the difference in z-axis reference and the difference in phase (x, y-axis plane)
    
    # Difference on the z-axis
    delta_z = abs((acos(target.z) - acos(node.z)) / 0.0365)
    
    # Phase Difference
    # Not accurate and needs to be corrected, but there is no significant change in the value.
    phase = target.phase - node.phase + 0.1726*(df_theta-acos(node.z))
    # phase = target.phase - node.phase + 0.1726*theta

    # Adjust the value when negative.
    if phase < 0:
        phase += 2*pi

    # Arbitrary weight 0.1
    phi = phase*0.1

    h = delta_z + phi

    return h

In [78]:
import ast

def before_main(target_theta, target_phi, predicted_comb):
     
     # Target state
     # Theta must move first and then phi move.
     target_U = Rz(target_phi) @ Rx(target_theta) 
     irho_target = target_U @ irho_init @ target_U.conj().T

     irho_mid = np.matrix(irho_init)
     
     for i in predicted_comb:
          instant_U = unitary(2.6, i)             # 2.6 dt 기존 predicted_comb 정보
          irho_mid = (instant_U @ irho_mid @ instant_U.conj().T)
     
     return irho_mid, irho_target


In [79]:
def main(target_theta, target_phi, predicted_comb) :
    
    irho_mid, irho_target = before_main(target_theta, target_phi, predicted_comb)

    # Finding path 
    start = Node(None, irho_mid, None)
    end = Node(None, irho_target, None)
    
    # Calculating the computational time for a pair of theta and phi
    start_time = time.time()
    
    path = Astar_Qsearch(start, end)
    
    end_time = time.time()
    
    # Calculating fidelity
    path_fidelity = state_fidelity(irho_mid, irho_target)
    
    # AStar Algorithm computation time
    computing_time = end_time - start_time
    
    # Actual dt applied time
    total_time = dt*len(path)

    # Result output
    print(f"""
    ------------------------------------------------------------------------------------------------------
    theta = {target_theta}
    phi = {target_phi}
    path : {path}
    fidelity : {path_fidelity}
    total_time : {total_time}
    computing_time : {computing_time}
    ------------------------------------------------------------------------------------------------------
    """)
    
    return path, path_fidelity, total_time, computing_time

In [80]:
import os
from datetime import datetime
import pandas as pd

# Create a directory to store the results
dir = 'Data_local_optimization_results_dir'
if not os.path.exists(dir):
    os.makedirs(dir)

# CSV file name setup
date = datetime.now()
printdate = date.strftime('%Y%m%d_%H%M%S')
filename = "/Local_Optimization_ByAstar_99.99_" + printdate + '.csv'

# Create an empty DataFrame and write to CSV file
df = pd.DataFrame(columns=['Case', 'gate length', 'Theta', 'Phi', 'dt', 
                           'DL combination', 'DL fidelity', 'DL real time', 'DL computing time', 
                           'A* combination', 'A* fidelity', 'A* real time', 'A* computing time', 
                           'Total combination', 'Total fidelity', 'Total real time', 'Total computing time'])

df.to_csv(dir + filename, index=False)

# upload existed file - Theta, Phi, Combination, Fidelity, Computing time
upload_df = pd.read_csv('data_results/NVspin_GRU_data_random_results_20230919_142458.csv')
sample = upload_df.loc[:, ["Theta", "Phi", 'dt', 'DL combination', 'DL fidelity', 'DL real time', 'DL computing time']]

print(sample)
len(upload_df)

         Theta       Phi   dt  \
0     2.376764  2.143035  2.6   
1     1.240419  4.057858  2.6   
2     2.611981  2.592080  2.6   
3     2.557084  3.359399  2.6   
4     0.673767  4.998533  2.6   
...        ...       ...  ...   
9995  2.436158  2.715927  2.6   
9996  1.739832  3.324668  2.6   
9997  1.356168  6.017059  2.6   
9998  2.274292  5.411439  2.6   
9999  0.746675  4.931662  2.6   

                                         DL combination  DL fidelity  \
0     [0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 2, 2, 4, 4, ...      0.99726   
1     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...      0.98423   
2     [0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 2, 2, 4, 4, ...      0.98394   
3     [0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 2, 2, 4, 4, 1, ...      0.99525   
4     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...      0.99425   
...                                                 ...          ...   
9995  [0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 2, 2, 4, 4, 1, ...      0.99396   
9996  [0, 0, 0, 0, 0, 0, 0,

10000

In [81]:
sample['DL combination'] = sample['DL combination'].apply(ast.literal_eval)

In [82]:
case = 0

for case in range(len(upload_df) - 1) : 
    
    df_theta = sample["Theta"][case]
    df_phi = sample["Phi"][case]
    df_dt = sample["dt"][case]
    df_comb = sample["DL combination"][case]
    df_fid = sample["DL fidelity"][case]
    df_tott = sample["DL real time"][case]
    df_compt = sample["DL computing time"][case]
    
    if df_fid < 99.99 :
        path, path_fid, path_tott, path_compt = main(df_theta, df_phi, df_comb)
    else:
        path = []
        path_fid = df_fid
        path_tott = 0
        path_compt = 0
    
    if path == [-1]:
        total_path = df_comb
        total_tott = df_tott
        total_fid = df_fid
    else:
        total_path = df_comb + path
        total_tott = df_tott + path_tott
        total_fid = fidelity(df_theta, df_phi, total_path)                  # 전체 경로 fidelity 확인
    
    total_compt = df_compt + path_compt
    
    output = [['case' + str(case), len(path), df_theta, df_phi, df_dt, 
               df_comb, df_fid, df_tott, df_compt,                                      # DL 기반 정보
               path, path_fid, path_tott, path_compt,                                   # A* 기반 정보
               total_path, total_fid, total_tott, total_compt]]                         # 최종 정보
    
    # Create DataFrame and append to CSV file
    df = pd.DataFrame(output, columns=[ "Case", 'gate length', 'Theta', 'Phi', 'dt', 
                                        'DL combination', 'DL fidelity', 'DL real time', 'DL computing time', 
                                        'A* combination', 'A* fidelity', 'A* real time', 'A* computing time', 
                                        'Total combination', 'Total fidelity', 'Total real time', 'Total computing time'])
    
    df.to_csv(dir + filename, mode='a', header=False, index=False)


0  = >  0 0 0
iter : 0  => 0.9972596885787816
1  = >  1.3 1.062661737808242 2.362661737808242
iter : 1  => 0.9646277582691578
2  = >  1.3 1.28352216305246 2.58352216305246
iter : 2  => 0.9740756564170385
3  = >  2.6 0.6444213404095314 3.2444213404095317
iter : 3  => 0.9111434414919463
4  = >  2.6 0.7574671657759552 3.3574671657759554
iter : 4  => 0.9257528334047672
5  = >  1.3 2.187200677297082 3.487200677297082
iter : 5  => 0.9695220254724451
6  = >  2.6 0.7854776450026684 3.3854776450026685
iter : 6  => 0.9054186827998784
7  = >  2.6 1.0236617378059507 3.623661737805951
iter : 7  => 0.9025018454100008
8  = >  2.6 1.1810794287704205 3.7810794287704206
iter : 8  => 0.9104975097403396
9  = >  2.6 1.244522163056847 3.844522163056847
iter : 9  => 0.9179885386930177
10  = >  2.6 1.3285488391092946 3.9285488391092946
iter : 10  => 0.8929767820906899
11  = >  2.6 1.4173151100276713 4.017315110027671
iter : 11  => 0.8945830048341373
12  = >  2.6 1.6594926315824634 4.2594926315824635
iter : 12

KeyboardInterrupt: 